In [228]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
import numpy as np
from sklearn.svm import SVC, SVR
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
    average_precision_score, 
    auc
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel

In [202]:
white_df = pd.read_csv("../data/winequality-white.csv", sep=";")
red_df = pd.read_csv("../data/winequality-red.csv", sep=";")

red_df['type']='red wine'
white_df['type']='white wine'
wine = pd.concat([red_df,white_df]).reset_index().drop(columns = ['index'])

We combined both csv files together and created a "type" column to see whether the type of the wine will affect the judgement of the quality. Because sometimes, human's perception of wine type may affect the independent scoring on the wine quality, so that's why we added a binary feature to account for this factor. 

In [203]:
train_df, test_df = train_test_split(wine, test_size=0.2, random_state=123)

In [272]:
X_train = train_df.drop(columns=["quality"])
X_test = test_df.drop(columns=["quality"])

y_train = train_df["quality"]
y_test = test_df["quality"]

In [205]:
numeric_feats = X_train.select_dtypes(include=[np.number]).columns.values.tolist()
binary_feats = ["type"]

numeric_transformer = make_pipeline(StandardScaler())
binary_transformer = make_pipeline(OneHotEncoder(drop="if_binary", dtype=int))

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_feats),
    (binary_transformer, binary_feats)
)

In our data file, we only have numerical features originally; however we need to scale those number before we feed them in the model. And we engineered a binary feature ("type"), we used `OneHotEncoder` with `drop="if_binary"` argument to form the transformer. 

In [206]:
column_name = numeric_feats + binary_feats
pd.DataFrame(preprocessor.fit_transform(X_train, y_train), columns = column_name)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,-1.552156,-0.537847,-0.616843,-0.849603,-0.256654,-0.599698,-0.688718,-1.550251,0.637496,0.602356,0.755569,1.0
1,-0.551023,-0.903607,-0.274431,-0.849603,-0.285082,-0.487984,-0.460217,-1.427678,-0.236615,-0.134424,1.005574,1.0
2,0.065059,-0.720727,0.547359,1.949924,-0.398798,0.768801,0.234073,1.692956,0.887242,-0.804224,-1.161136,1.0
3,-0.165971,-0.598807,0.273429,-0.556020,-0.626230,-0.487984,-0.073524,-1.553564,-0.174178,-1.005164,1.755589,1.0
4,-0.011951,-0.598807,-0.137466,-0.807663,-0.228225,-0.208698,0.260438,-0.460348,0.200441,-0.536304,0.005554,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5192,-0.319992,0.681350,-0.274431,4.319561,-0.711516,-0.208698,0.102245,2.736480,-0.985853,-0.737244,0.088889,1.0
5193,0.758151,-0.476888,0.067982,-0.597961,-0.086080,-1.102412,-0.794180,-0.221828,-2.047274,-0.268384,-0.827796,1.0
5194,-0.859063,1.534788,-2.123458,-0.702812,-0.000793,-1.437555,-1.813644,0.010066,1.886227,0.200476,0.755569,0.0
5195,0.604131,-0.720727,-0.274431,1.792647,-0.086080,2.919299,1.420518,1.129784,-0.486361,-0.536304,-0.577791,1.0


In [207]:
# Imported from DSCI 573 Lecture Notes from UBC
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [233]:
# Imported from DSCI 573 Lecture Notes from UBC
def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))


# make a scorer function that we can pass into cross-validation
mape_scorer = make_scorer(mape, greater_is_better=False)

score_types_reg = {
    "neg_mean_squared_error": "neg_mean_squared_error",
    "neg_root_mean_squared_error": "neg_root_mean_squared_error",
    "neg_mean_absolute_error": "neg_mean_absolute_error",
    "r2": "r2",
    "mape_scorer": mape_scorer,
}

models = {
    "Ridge": Ridge(max_iter=2000),
    "SVC": SVC(),
    "OneVsRest":OneVsRestClassifier(LogisticRegression()),
    "Random Forest": RandomForestRegressor(random_state=123)
}

In [234]:
results_comb={}
for keys in models.keys():
    pipe_comb = make_pipeline(preprocessor, models[keys])
    results_comb[keys]=mean_std_cross_val_scores(
        pipe_comb, X_train, y_train, return_train_score=True, scoring=score_types_reg
    )
pd.DataFrame(results_comb)

,Ridge,SVC,OneVsRest,Random Forest
fit_time,0.010 (+/- 0.002),0.770 (+/- 0.051),0.100 (+/- 0.006),1.501 (+/- 0.023)
score_time,0.005 (+/- 0.001),0.373 (+/- 0.008),0.006 (+/- 0.000),0.021 (+/- 0.000)
test_neg_mean_squared_error,-0.542 (+/- 0.034),-0.587 (+/- 0.016),-0.648 (+/- 0.006),-0.395 (+/- 0.027)
train_neg_mean_squared_error,-0.536 (+/- 0.009),-0.542 (+/- 0.007),-0.642 (+/- 0.005),-0.056 (+/- 0.002)
test_neg_root_mean_squared_error,-0.736 (+/- 0.023),-0.766 (+/- 0.010),-0.805 (+/- 0.004),-0.628 (+/- 0.021)
train_neg_root_mean_squared_error,-0.732 (+/- 0.006),-0.736 (+/- 0.005),-0.801 (+/- 0.003),-0.237 (+/- 0.003)
test_neg_mean_absolute_error,-0.570 (+/- 0.015),-0.480 (+/- 0.010),-0.520 (+/- 0.003),-0.450 (+/- 0.012)
train_neg_mean_absolute_error,-0.567 (+/- 0.004),-0.441 (+/- 0.004),-0.514 (+/- 0.002),-0.169 (+/- 0.002)
test_r2,0.289 (+/- 0.029),0.231 (+/- 0.020),0.150 (+/- 0.006),0.482 (+/- 0.027)
train_r2,0.298 (+/- 0.007),0.290 (+/- 0.011),0.158 (+/- 0.008),0.926 (+/- 0.002)


After comparing different regression models by using various matrix, we found the better model is Random Forest, because we got highest cross-validation score. However, we figured out that we may encounter some overfitting issue with Random Forest model as the difference between train score and validation score is quite wide. So, we further conduct feature selections and hyper-parameter optimization as follows. 

In [258]:
rfe = RFE(RandomForestRegressor(random_state=123), n_features_to_select=10)

pipe_rf_rfe = make_pipeline(preprocessor, rfe, RandomForestRegressor(random_state=123))

results_comb['Random Forest_rfe'] = mean_std_cross_val_scores(pipe_rf_rfe, X_train, y_train, return_train_score=True, scoring=score_types_reg)

In [259]:
pd.DataFrame(results_comb)

,Ridge,SVC,OneVsRest,Random Forest,Random Forest_rfe
fit_time,0.010 (+/- 0.002),0.770 (+/- 0.051),0.100 (+/- 0.006),1.501 (+/- 0.023),5.762 (+/- 0.081)
score_time,0.005 (+/- 0.001),0.373 (+/- 0.008),0.006 (+/- 0.000),0.021 (+/- 0.000),0.021 (+/- 0.000)
test_neg_mean_squared_error,-0.542 (+/- 0.034),-0.587 (+/- 0.016),-0.648 (+/- 0.006),-0.395 (+/- 0.027),-0.395 (+/- 0.031)
train_neg_mean_squared_error,-0.536 (+/- 0.009),-0.542 (+/- 0.007),-0.642 (+/- 0.005),-0.056 (+/- 0.002),-0.056 (+/- 0.002)
test_neg_root_mean_squared_error,-0.736 (+/- 0.023),-0.766 (+/- 0.010),-0.805 (+/- 0.004),-0.628 (+/- 0.021),-0.628 (+/- 0.025)
train_neg_root_mean_squared_error,-0.732 (+/- 0.006),-0.736 (+/- 0.005),-0.801 (+/- 0.003),-0.237 (+/- 0.003),-0.238 (+/- 0.003)
test_neg_mean_absolute_error,-0.570 (+/- 0.015),-0.480 (+/- 0.010),-0.520 (+/- 0.003),-0.450 (+/- 0.012),-0.451 (+/- 0.015)
train_neg_mean_absolute_error,-0.567 (+/- 0.004),-0.441 (+/- 0.004),-0.514 (+/- 0.002),-0.169 (+/- 0.002),-0.169 (+/- 0.002)
test_r2,0.289 (+/- 0.029),0.231 (+/- 0.020),0.150 (+/- 0.006),0.482 (+/- 0.027),0.481 (+/- 0.032)
train_r2,0.298 (+/- 0.007),0.290 (+/- 0.011),0.158 (+/- 0.008),0.926 (+/- 0.002),0.926 (+/- 0.002)


In [260]:
pipe_rf_rfe.fit(X_train, y_train)
pipe_rf_rfe.named_steps['rfe'].ranking_

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3])

In [261]:
rfe_fs = pipe_rf_rfe.named_steps["rfe"].support_
rfe_fs

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False])

In [262]:
rfe_selected_feats = X_train.columns[rfe_fs]
rfe_selected_feats

Index(['volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol'],
      dtype='object')

By using Recursive Feature Elimination algorithm, we chose to drop "type" and "fixed acidity" features. Because by dropping those two features, even though we did not get better scores, we are able to achieve the same scores with lesser features. It is great because it will simply our model and save cost for future data collection. 

In [263]:
from scipy.stats import randint

In [273]:
param_dist = {"randomforestregressor__max_depth": randint(low=5, high=1000),
             "randomforestregressor__max_leaf_nodes": randint(low=5, high=1000),
             "randomforestregressor__n_estimators": randint(low=5, high=1000),}

random_search = RandomizedSearchCV(
    pipe_rf_rfe,
    param_distributions=param_dist,
    n_jobs=-1,
    n_iter=10,
    cv=5,
    random_state=123
)

random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['fixed '
                                                                                'acidity',
                                                                                'volatile '
                                                                                'acidity',
                                                                                'citric '
                                                                                'acid',
                                           

In [274]:
random_search.best_params_

{'randomforestregressor__max_depth': 344,
 'randomforestregressor__max_leaf_nodes': 851,
 'randomforestregressor__n_estimators': 258}

In [275]:
random_search.best_score_

0.48263333317966534

In [276]:
random_search.score(X_train, y_train)

0.913462954551246

In [277]:
random_search.score(X_test, y_test)

0.5322298294597634

Finally, we conduct hyper-parameter optimization. We employed random search to look for same key hyper-parameters, such as max_depth, max_leaf_nodes, and n_estimators. The best hyper-parameter value we got from the algorithm are ['max_depth': 344, 'max_leaf_nodes': 851, 'n_estimators': 258]. And the best validation score we got is 0.48. And the test score is 0.52 after tunning the hyper-parameters. However, as we discovered above, the train score is 0.91, which means we still have overfitting issue by using Random Forest model. We may investigate further and figure out the next steps later. 